In [1]:
import sys
print(sys.version)

3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]


In [20]:
import os
from google.colab import drive
import numpy as np
import glob
import json

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from typing import Tuple, List, Dict, Iterable
from skimage.io import imread
from skimage.transform import resize
import logging
import math
from typing import Optional
import random
from pathlib import Path
import h5py

from typing import NamedTuple, Tuple, List

%matplotlib inline

In [ ]:
!pip install requests
!pip install unrar

In [ ]:
%load_ext tensorboard

In [3]:
log = logging.getLogger(__name__)

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%%capture
!unrar x -Y "/content/drive/My Drive/Ilmenau/dataset.rar" "/tmp/"

In [37]:
INPUT_SHAPE = (1280, 960, 3)
BATCH_SIZE = 32
AMOUNT_OF_FRAMES = 500
VALIDATION_SPLIT = 0.2
MAX_LINES_PER_FRAME = 2
MAX_NUM_POINTS =  91
NUM_TYPE_OF_LINES = 4

In [26]:
BASE_DIR = "/tmp/dataset/VIL100/"
IMAGE_PATH = BASE_DIR + "JPEGImages/"
JSON_PATH = BASE_DIR + "Json/"
JSON_HDF5_DATASET_PATH = BASE_DIR + "hdf5/"

print(IMAGE_PATH)
print(JSON_PATH)
print(JSON_HDF5_DATASET_PATH)

/tmp/dataset/VIL100/JPEGImages/
/tmp/dataset/VIL100/Json/
/tmp/dataset/VIL100/hdf5/


In [8]:
images = glob.glob(IMAGE_PATH+'/*/*.jpg')
json_files = glob.glob(JSON_PATH+'/*/*.json')
json_glob_path = JSON_PATH + '/*/*.json'

In [13]:
class VIL100HDF5:
    ROOT_FOLDER = 'hdf5'
    GROUP_NAME = 'frame_polylines_labels'
    POLYLINES_DATASET_NAME = 'polylines'
    LABELS_DATASET_NAME = 'labels'
LANE_ID_FULL_LIST = set(range(1, 3))


class Vil100Json:
    ANNOTATIONS = 'annotations'
    ATTRIBUTE = 'attribute'
    LANE = 'lane'
    LANE_ID = 'lane_id'
    POINTS = 'points'


class VIL100Attribute:
    """Lane Attribute id (type lane) in jsons"""
    SINGLE_WHITE_SOLID = 1
    SINGLE_WHITE_DOTTED = 2
    SINGLE_YELLOW_SOLID = 3
    SINGLE_YELLOW_DOTTED = 4
    DOUBLE_WHITE_SOLID = 5
    DOUBLE_YELLOW_SOLID = 7
    DOUBLE_YELLOW_DOTTED = 8
    DOUBLE_WHITE_SOLID_DOTTED = 9
    DOUBLE_WHITE_DOTTED_SOLID = 10
    DOUBLE_SOLID_WHITE_AND_YELLOW = 13


class LineType:
    """Type lane in our task"""
    NO_LINE = 0
    SINGLE_WHITE_SOLID = 1
    SINGLE_WHITE_DOTTED = 2
    DOUBLE_WHITE_SOLID = 3
    ALL_LINES = {NO_LINE, SINGLE_WHITE_SOLID, SINGLE_WHITE_DOTTED, DOUBLE_WHITE_SOLID}


VIL_100_colour_line = {
    LineType.SINGLE_WHITE_SOLID: (255, 0, 0),  # single white solid
    LineType.SINGLE_WHITE_DOTTED: (0, 255, 0),  # single white dotted
    LineType.DOUBLE_WHITE_SOLID: (255, 125, 0),  # single yellow solid
    # 4: (255, 255, 0),  # single yellow dotted
    # 5: (255, 0, 0),  # double white solid
    # 6: (255, 125, 0),  # double yellow solid
    # 7: (255, 255, 0),  # double yellow dotted
    # 8: (255, 0, 0),  # double white solid dotted
    # 9: (255, 0, 0),  # double white dotted solid
    # 10: (255, 0, 0),  # double solid white and yellow
}


def get_valid_attribute(attr: int) -> int:
    """Change attribute from VIL100 dataset to normal number without missings"""
    _VIL_100_attributes = {
        LineType.NO_LINE: LineType.NO_LINE,
        VIL100Attribute.SINGLE_WHITE_SOLID: LineType.SINGLE_WHITE_SOLID,
        VIL100Attribute.SINGLE_WHITE_DOTTED: LineType.SINGLE_WHITE_DOTTED,
        VIL100Attribute.SINGLE_YELLOW_SOLID: LineType.SINGLE_WHITE_SOLID,
        VIL100Attribute.SINGLE_YELLOW_DOTTED: LineType.SINGLE_WHITE_DOTTED,
        VIL100Attribute.DOUBLE_WHITE_SOLID: LineType.DOUBLE_WHITE_SOLID,
        VIL100Attribute.DOUBLE_YELLOW_SOLID: LineType.DOUBLE_WHITE_SOLID,
        VIL100Attribute.DOUBLE_YELLOW_DOTTED: LineType.DOUBLE_WHITE_SOLID,
        VIL100Attribute.DOUBLE_WHITE_SOLID_DOTTED: LineType.DOUBLE_WHITE_SOLID,
        VIL100Attribute.DOUBLE_WHITE_DOTTED_SOLID: LineType.DOUBLE_WHITE_SOLID,
        VIL100Attribute.DOUBLE_SOLID_WHITE_AND_YELLOW: LineType.DOUBLE_WHITE_SOLID,
    }
    return _VIL_100_attributes.get(attr, LineType.NO_LINE)


def get_colour_from_one_hot_vector(vector: np.ndarray) -> Tuple[int, int, int]:
    """Get colour from one hot vector"""
    return VIL_100_colour_line.get(int(np.argmax(vector, axis=1)), None)


def one_hot_list_encoder(target_class_idx: int, num_classes: int) -> np.ndarray:
    """One-hot list encoder"""
    target_vector = np.zeros(num_classes)
    target_vector[target_class_idx] = 1
    return target_vector

In [14]:
class VILLJsonConverter:

    def __init__(self,
                 max_lines_per_frame: int,
                 max_num_points: int,
                 num_type_of_lines: int,
                 json_glob_path: str = None, ):

        self.max_lines_per_frame = max_lines_per_frame
        self.max_num_points = max_num_points
        self.num_type_of_lines = num_type_of_lines
        self.json_files = sorted(glob.glob(json_glob_path))
        self.files_count = len(self.json_files)

    def __get_polyline_with_label(self, lane: dict) -> Tuple[np.ndarray, np.ndarray]:
        """Get array from points list"""
        points = np.array(
            lane[Vil100Json.POINTS]).flatten()
        points = np.pad(points, pad_width=(0, self.max_num_points * 2 - points.shape[0]))
        # TODO @Karim: remember below `label.get(label)` is index 1,2,3,4
        label = get_valid_attribute(lane.get(Vil100Json.ATTRIBUTE, LineType.NO_LINE))
        labels = one_hot_list_encoder(label, self.num_type_of_lines)
        return points, labels

    def __get_polyline_and_label_from_file(self, json_path: str) -> Tuple[np.ndarray, np.ndarray]:
        """
        Retrieve from json file polylines and labels and format to nn input

        :param json_path: json file path
        :return: frame and tuple of labels
        """
        with open(json_path) as f:
            lanes: List[Dict[str, int]] = json.load(f)[Vil100Json.ANNOTATIONS][Vil100Json.LANE]
            lanes = sorted(lanes, key=lambda lane: lane[Vil100Json.LANE_ID])

            if lanes:
                polylines, labels = np.array([]), np.array([])
                # TODO @Karim: check another params in json files like "occlusion"
                exist_lane = [x[Vil100Json.LANE_ID] for x in lanes]
                missed_lane = LANE_ID_FULL_LIST - set(exist_lane)

                for lane_id in range(1, self.max_lines_per_frame + 1):
                    if lane_id in missed_lane:
                        points = np.zeros(shape=(self.max_num_points * 2))
                        label = one_hot_list_encoder(LineType.NO_LINE, self.num_type_of_lines)
                    else:
                        points, label = self.__get_polyline_with_label(lane=lanes[exist_lane.index(lane_id)])

                    if lane_id % 2 == 0:
                        polylines = np.append(polylines, points)
                        labels = np.append(labels, label)
                    else:
                        polylines = np.insert(polylines, 0, points)
                        labels = np.insert(labels, 0, label)

                return polylines, labels
            else:
                empty_label = one_hot_list_encoder(LineType.NO_LINE, self.num_type_of_lines)
                polylines_empty_shape = self.max_lines_per_frame * self.max_num_points * 2
                return np.zeros(shape=polylines_empty_shape), np.array(
                    [empty_label for x in range(self.max_lines_per_frame)]).flatten()

    def exec(self) -> None:
        """Convert and save json files to new hdf5 files"""
        for json_file_path in self.json_files:
            polylines, labels = self.__get_polyline_and_label_from_file(json_file_path)

            full_path_list = json_file_path.split('/')
            full_path_list[-3] = VIL100HDF5.ROOT_FOLDER
            root_path = full_path_list[:-1]
            frame_name = full_path_list[-1]

            Path(f"{'/'.join(root_path)}").mkdir(parents=True, exist_ok=True)

            with h5py.File(f"{'/'.join(root_path)}/{frame_name}.hdf5", "w") as f:
                grp = f.create_group(VIL100HDF5.GROUP_NAME)
                grp.create_dataset(VIL100HDF5.POLYLINES_DATASET_NAME, data=polylines, dtype='int32')
                grp.create_dataset(VIL100HDF5.LABELS_DATASET_NAME, data=labels, dtype='int32')


In [15]:
converter = VILLJsonConverter(
        max_lines_per_frame=MAX_LINES_PER_FRAME,
        max_num_points=MAX_NUM_POINTS,
        num_type_of_lines=NUM_TYPE_OF_LINES,
        json_glob_path=json_glob_path,
    )

In [21]:
converter.exec()

In [22]:
def calculate_perspective_transform_matrix(width: int, height: int, reverse_flag=False) -> Tuple[
    np.ndarray]:
    """
    Calculate transformation matrix for perspective transformation
    :param width: frame width
    :param height: frame height
    :param reverse_flag: create reverse matrix for reverting to initial frame
    :return: matrix for transformation the frame
    """
    # TODO @Karim: check on real Audi Q2 input frame
    high_left_crd, high_right_crd = (550, 530), (700, 530)
    down_left_crd, down_right_crd, = (0, height - 150), (width, height - 150)

    initial_matrix = np.float32([[high_left_crd, high_right_crd,
                                  down_left_crd, down_right_crd]])
    final_matrix = np.float32([[(0, 0), (width, 0), (0, height), (width, height)]])

    return cv2.getPerspectiveTransform(initial_matrix, final_matrix) \
        if not reverse_flag else cv2.getPerspectiveTransform(final_matrix, initial_matrix)


def transform_frame(frame: np.ndarray, width: int, height: int, reverse_flag=False) -> np.ndarray:
    """
    Perform perspective transformation
    :param frame: frame
    :param width: frame width
    :param height: frame height
    :param reverse_flag: cancel perspective transformation
    :return: changed (un)transformed frame
    """
    if not reverse_flag:
        initial_matrix = calculate_perspective_transform_matrix(width, height)
        frame = cv2.warpPerspective(frame, initial_matrix, dsize=(width, height))
    else:
        final_matrix = calculate_perspective_transform_matrix(width, height, reverse_flag=True)
        frame = cv2.warpPerspective(frame, final_matrix, dsize=(width, height))
    return frame

In [27]:
class SimpleFrameGenerator(Sequence):
    """Sequence of frames generator

    Usage for training NN that could process independent
    frames without context window etc
    """

    def __init__(self,
                 num_type_of_lines=4,
                 max_num_points=91,
                 max_lines_per_frame=2,
                 rescale=1 / 255.,  # TODO @Karim: include and use later
                 batch_size: int = 8,
                 target_shape: Tuple[int, int] = (1280, 960),
                 shuffle: bool = False,
                 nb_channel: int = 3,  # TODO: Use rgb later
                 files: Optional[List[str]] = None,
                 json_files: Optional[List[str]] = None):
        """
        :param subset: training or validation data
        :param max_lines_per_frame: maxinum number of lines per frame
        :param max_num_points: maximum number of points un one polyline
        :param num_type_of_lines: number of possible lines on road
        :param rescale:
        :param batch_size: batch size of the dataset
        :param target_shape: final size for NN input
        :param shuffle: shuffle flag of frames sequences
        :param split: split dataset to train/test
        :param nb_channel: grayscaled or RGB frames
        :param frame_glob_path: glob pattern of frames
        :param json_glob_path: glob pattern path of jsons
        """
        self.max_lines_per_frame = max_lines_per_frame
        self.max_num_points = max_num_points
        self.num_type_of_lines = num_type_of_lines
        self.rescale = rescale
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_shape = target_shape
        self.nb_channel = nb_channel
        self.files = files
        self.json_files = json_files
        self.files_count = len(self.files)

        if shuffle:
            temp = list(zip(self.files, self.json_files))
            random.shuffle(temp)
            self.files, self.json_files = zip(*temp)

    def __len__(self):
        return math.ceil(self.files_count / self.batch_size)

    def __get_polyline_and_label_from_file(self, json_path: str) -> Tuple[np.ndarray, np.ndarray]:
        """
        Get from hdf5 all polylines and their labels
        :param json_path: path of json file
        :return: polylines with labels
        """
        file = h5py.File(json_path, 'r')
        group = file.get(VIL100HDF5.GROUP_NAME)
        return group.get(VIL100HDF5.POLYLINES_DATASET_NAME), group.get(VIL100HDF5.LABELS_DATASET_NAME)

    def __getitem__(self, idx) -> Tuple[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
        batch_frames_path = self.files[idx * self.batch_size:
                                       (idx + 1) * self.batch_size]
        batch_json_path = self.json_files[idx * self.batch_size:
                                          (idx + 1) * self.batch_size]

        polylines_list, labels_list = self.__get_polyline_and_label_from_file(batch_json_path[0])
        for _json in batch_json_path[1:]:
            polylines, labels = self.__get_polyline_and_label_from_file(_json)
            polylines_list = np.vstack((polylines_list, polylines))
            labels_list = np.vstack((labels_list, labels))

        return np.array([
            resize(imread(file_name) * self.rescale, self.target_shape) for file_name in
            batch_frames_path]), (polylines_list,) + tuple(np.hsplit(labels_list, self.max_lines_per_frame))


class SimpleFrameDataGen:
    TRAINING = 'training'
    VALIDATION = 'validation'

    __reverse_dataset_type = {
        TRAINING: VALIDATION,
        VALIDATION: TRAINING
    }
    __dataset = {}

    def __init__(self,
                 rescale=1 / 255.,
                 validation_split: Optional[float] = None,
                 frame_glob_path: str = "",
                 json_hdf5_glob_path: str = ""):
        """
        :param validation_split: split for train/validation sets
        :param rescale:
        :param frame_glob_path: glob pattern of frames
        :param json_glob_path: glob pattern path of jsons
        """
        self.rescale = rescale
        self.validation_split = validation_split

        self.__frame_glob_path = frame_glob_path
        self.__json_hdf5_glob_path = json_hdf5_glob_path

    def flow_from_directory(self, subset: str = TRAINING,
                            shuffle: bool = True, number_files: int = 2000, *args, **kwargs) -> SimpleFrameGenerator:
        """
        Get generator for subset
        :param subset: 'training' or 'validation'
        :param shuffle: flag for shuffling
        :param number_files: rectrict max number of files from dataset
        :param args: args for specific dataset
        :param kwargs: kwargs for specific dataset
        :return: Specific generator for specific subset
        """

        files = sorted(glob.glob(self.__frame_glob_path))
        log.info(f"Number of files in dataset: {len(files)}. Using in training/validation: {number_files}")
        files = files[:number_files]

        json_files = sorted(glob.glob(self.__json_hdf5_glob_path))[:number_files]
        files_count = len(files)
        json_files_count = len(json_files)

        if files_count != json_files_count:
            log.error(f"Dataset files error"
                      f"Number of frames: ({files_count}). "
                      f"Number of jsons({json_files_count}")
            raise FileNotFoundError(
                f"Numbers of frames and jsons are not equal!")

        if not self.__reverse_dataset_type.get(subset):
            log.error(f'Wrong subset value: "{subset}"')
            raise ValueError(f'Wrong type of subset - {subset}. '
                             f'Available types: {self.__reverse_dataset_type.keys()}')

        if self.validation_split and 0.0 < self.validation_split < 1.0:
            split = int(files_count * (1 - self.validation_split))
            if subset == self.TRAINING:
                files = files[:split]
                json_files = json_files[:split]
            else:
                files = files[split:]
                json_files = json_files[split:]

        return SimpleFrameGenerator(rescale=self.rescale,
                                    files=files,
                                    shuffle=shuffle,
                                    json_files=json_files,
                                    *args, **kwargs)


In [38]:
data_gen = SimpleFrameDataGen(
    validation_split=VALIDATION_SPLIT, 
    frame_glob_path=IMAGE_PATH+'/*/*.jpg', 
    json_hdf5_glob_path=JSON_HDF5_DATASET_PATH+'/*/*.hdf5',
)

train_generator = data_gen.flow_from_directory(
  subset='training', shuffle=True, batch_size = BATCH_SIZE, 
  number_files=AMOUNT_OF_FRAMES, max_lines_per_frame=MAX_LINES_PER_FRAME,
  max_num_points = MAX_NUM_POINTS, num_type_of_lines = NUM_TYPE_OF_LINES
)

validation_generator = data_gen.flow_from_directory(
  subset='validation', shuffle=True, batch_size = BATCH_SIZE, 
  number_files=AMOUNT_OF_FRAMES, max_lines_per_frame=MAX_LINES_PER_FRAME,
  max_num_points = MAX_NUM_POINTS, num_type_of_lines = NUM_TYPE_OF_LINES
)


In [ ]:
# for image_polylines in validation_generator:
#     print(image_polylines[0].shape)
#     print(image_polylines[1][0].shape)
#     print(image_polylines[1][1].shape)
#     break


In [30]:
from tensorflow.keras import layers, Model

model_name = 'lane_line_cnn_model'

def build_model(polyline_output_shape:int ,label_output_shape:int, input_shape=(1280, 960, 3)):

    # pretrained
    pre_trained_model = tf.keras.applications.InceptionResNetV2(input_shape=input_shape,
                            weights='imagenet',
                            include_top=False)  
    global_max_pool = layers.GlobalMaxPool2D()(pre_trained_model.output)
    dropout_max_pool = layers.Dropout(.2)(global_max_pool)
    
    # polyline part
    dense_polyline = tf.keras.layers.Dense(units=512, activation='relu')(dropout_max_pool)
    dropout_polyine = layers.Dropout(.2)(dense_polyline)
    dense_polyline_2 = tf.keras.layers.Dense(units=512, activation='relu')(dropout_polyine)
    dropout_polyine_2 = layers.Dropout(.2)(dense_polyline_2)

    # label common part
    dense_label = tf.keras.layers.Dense(units=256, activation='relu')(dropout_max_pool)
    dropout_label = layers.Dropout(.2)(dense_label)

    # lane 1 part
    x = tf.keras.layers.Dense(units=128, activation='relu')(dropout_label)
    x = layers.Dropout(.2)(x)
    x = tf.keras.layers.Dense(units=64, activation='relu')(x)
    x = layers.Dropout(.2)(x)
    x = tf.keras.layers.Dense(units=32, activation='relu')(x)
    x = layers.Dropout(.2)(x)
   
    # lane 2 part
    y = tf.keras.layers.Dense(units=128, activation='relu')(dropout_label)
    y = layers.Dropout(.2)(y)
    y = tf.keras.layers.Dense(units=64, activation='relu')(y)
    y = layers.Dropout(.2)(y)
    y = tf.keras.layers.Dense(units=32, activation='relu')(y)
    y = layers.Dropout(.2)(y)

    # output
    polyline_output = layers.Dense(polyline_output_shape,name='polyline_output')(dropout_polyine_2)
    label_output_1 = layers.Dense(label_output_shape, activation='softmax', name='label_output_1')(x)
    label_output_2 = layers.Dense(label_output_shape, activation='softmax', name='label_output_2')(y)

    model = Model(pre_trained_model.input, outputs=[
        polyline_output,
        label_output_1,
        label_output_2,
      ], name = model_name
    )

    return model, pre_trained_model

In [31]:
logdir = f"logs/{model_name}"
checkpoint_filepath = f"/model/{model_name}"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
early_stop_polyline_callback = tf.keras.callbacks.EarlyStopping(patience=6, monitor='val_polyline_output_loss')

reduce_lr_callback_depends_on_polyline_loss = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_polyline_output_loss', factor=0.7, patience=2, verbose=1, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0.00001
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_polyline_output_loss',
    mode='max',
    save_best_only=True)

In [32]:
model, pre_trained_model = build_model(
    polyline_output_shape=MAX_NUM_POINTS * 2 * MAX_LINES_PER_FRAME, 
    label_output_shape=NUM_TYPE_OF_LINES, 
    input_shape = INPUT_SHAPE)
# print(model.summary())

219070464/219055592 [==============================] - 6s 0us/step


In [33]:
pre_trained_model.trainable = False
# print(model.summary())

In [34]:
# tf.experimental.numpy.experimental_enable_numpy_behavior()
# tf.keras.utils.plot_model(model, "multi_output_model.png", show_shapes=True)

In [35]:
from tensorflow.keras.optimizers import Adam

learning_rate = 0.001

model.compile(loss= {
      'polyline_output':tf.keras.losses.MeanSquaredLogarithmicError(),
      'label_output_1':tf.keras.losses.CategoricalCrossentropy(),
      'label_output_2':tf.keras.losses.CategoricalCrossentropy(),
    },
    optimizer=Adam(learning_rate=learning_rate),
    metrics={'polyline_output':tf.keras.metrics.MeanSquaredLogarithmicError(),
             'label_output_1':'accuracy',
             'label_output_2':'accuracy',
             },
    loss_weights={
        "polyline_output": 500, 
        "label_output_1": 0.1,
        "label_output_2": 0.1,
      },)

In [39]:
history = model.fit(train_generator,
                    epochs=20,
                    verbose=2,
                    validation_data=validation_generator,
                    callbacks=[
                        tensorboard_callback,
                        early_stop_polyline_callback,
                        reduce_lr_callback_depends_on_polyline_loss,
                        model_checkpoint_callback,
                      ],
                    )

Epoch 1/20
13/13 - 220s - loss: 2089.3145 - polyline_output_loss: 4.1785 - label_output_1_loss: 0.0956 - label_output_2_loss: 0.7441 - polyline_output_mean_squared_logarithmic_error: 4.1785 - label_output_1_accuracy: 0.9650 - label_output_2_accuracy: 0.8275 - val_loss: 1490.0229 - val_polyline_output_loss: 2.9728 - val_label_output_1_loss: 36.1039 - val_label_output_2_loss: 0.0000e+00 - val_polyline_output_mean_squared_logarithmic_error: 2.9728 - val_label_output_1_accuracy: 0.0000e+00 - val_label_output_2_accuracy: 1.0000 - lr: 0.0010 - 220s/epoch - 17s/step
Epoch 2/20


KeyboardInterrupt: ignored

In [ ]:
# loss, polyline_output_loss,label_output_loss, polyline_output_mean_squared_logarithmic_error, label_output_accuracy = model.evaluate(validation_generator)
# print(f" \
#   Loss:{loss}\n \
#   polyline_output_loss:{polyline_output_loss}\n \
#   label_output_loss:{label_output_loss}\n \
#   polyline_output_mean_squared_logarithmic_error:{polyline_output_mean_squared_logarithmic_error}\n \
#   label_output_accuracy:{label_output_accuracy}"
# )

In [ ]:
%tensorboard --logdir logs/

In [ ]:
# !mkdir -p saved_model
# model.save('saved_model/my_model')

In [ ]:
from google.colab import files

In [ ]:
# files.download("saved_model/my_model")

In [ ]:
model_weight_name = f'{model_name}-weights.h5'

In [ ]:
# model.save(model_name) 

In [ ]:
# files.download(model_name)

In [ ]:
model.save_weights(model_weight_name)

In [ ]:
files.download(model_weight_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
res = model.predict(validation_generator[0][0])

In [ ]:
# res[0][0]

In [ ]:
res[1][0]

array([2.6892014e-03, 7.5523233e-01, 2.4188313e-01, 1.9534070e-04],
      dtype=float32)

In [ ]:
res[2][0]

array([0.00170986, 0.48654553, 0.5039854 , 0.0077592 ], dtype=float32)